In [84]:
import numpy as np
import pandas as pd
#for reading in data properly
import ast
import json

import gensim
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn import utils

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

In [85]:
all_data = pd.read_csv('train.csv')
all_data = all_data.dropna(subset=['overview', 'genres']) #drop cols without overview or genre (data we use or labels)

In [86]:
def text_to_list(x):
    if pd.isna(x):
        return ''
    else:
        return ast.literal_eval(x)

def parse_json(x):
    try:
        return json.loads(x.replace("'", '"'))[0]['name']
    except:
        return ''
    
def parse_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = [0]*len(genre_dict) #number of genres we are looking at
        for i in range(numElems):
            genre_str = (json_genres[i]['name'])
            if genre_str in genre_map.keys():
                ret[genre_dict[genre_map[genre_str]]] = 1
        return ret
    except Exception as excep:
        print('Exception' + str(excep))
        return ''

In [87]:
genre_dict = {}
genre_dict['Action-Adventure'] = 0
genre_dict['Romance'] = 1
genre_dict['Horror-Thriller'] = 2
genre_dict['Comedy'] = 3
genre_dict['Science Fiction'] = 4
#genre_dict['Drama'] = 5
genre_dict

{'Action-Adventure': 0,
 'Romance': 1,
 'Horror-Thriller': 2,
 'Comedy': 3,
 'Science Fiction': 4}

In [88]:
genre_map = {}
genre_map['Adventure'] = 'Action-Adventure'
genre_map['Romance'] = 'Romance'
genre_map['Horror'] = 'Horror-Thriller'
genre_map['Thriller'] = 'Horror-Thriller'
genre_map['Comedy'] = 'Comedy'
#genre_map['War'] = 'Action-Adventure'#not sure about this
genre_map['Action'] = 'Action-Adventure'
genre_map['Science Fiction'] = 'Science Fiction'
#genre_map['Drama'] = 'Drama'

In [89]:
def getGenresVects():
    y = all_data['genres']
    ret = y.apply(parse_genres_json)
    all_data['genres_vect'] = ret
    return ret

In [90]:
labels_vects = getGenresVects() #get label vectors for genres indexed by indexes in genre_dict

In [91]:
#put to lower case, remove punctation
def cleanText(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    text = ' '.join(no_stopword_text)
    text = re.sub(r'[^a-z A-Z0-9]', "", text) #maybe shouldn't remove punction between words here?
    text = text.lower()
    return text
all_data['cleanOverview'] = all_data['overview'].apply(cleanText)

In [92]:
all_data = all_data[all_data.genres_vect.map(sum) > 0]

In [93]:
#logistic regression data
lr_data = all_data[['cleanOverview', 'genres_vect', 'overview']]

In [94]:
train, test = train_test_split(lr_data, test_size=0.2, random_state=42)

CNN STUFF here

In [95]:
#get word embeddings
x = train['cleanOverview'].values.tolist()
y = train['genres_vect']

In [96]:
x_test = test['cleanOverview'].values.tolist()
y_test = test['genres_vect']

In [97]:
y_train = y.tolist()
y_train = np.array(y_train)

In [98]:
y_test = y_test.tolist()
y_test = np.array(y_test)

In [99]:
tok = [word_tokenize(sent) for sent in x]

In [100]:
word_vec_len = 32
model = Word2Vec(tok, min_count = 2, size=word_vec_len)

In [101]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words_kept = 100000 #using 100000 most popular words, use throughout

tokenizer = Tokenizer(num_words_kept)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

max_seq_len = 150

x_train_seq = pad_sequences(sequences, maxlen=max_seq_len)

In [102]:
test_sequences = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(test_sequences, maxlen=max_seq_len)

In [103]:
embeddings_index = {}
for w in model.wv.vocab.keys():
    embeddings_index[w] = model.wv[w]


embedding_matrix = np.zeros((num_words_kept, word_vec_len))
for word, i in tokenizer.word_index.items():
    if i >= num_words_kept:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [104]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def get_per_label_metrics(real_labels_matrix, predictions_labels_matrix):
    for genre in genre_dict.keys():
        index = genre_dict[genre]
        real_labels_vect = real_labels_matrix[:, index]
        prediction_vect = predictions_labels_matrix[:,index]
        print("Accuruacy for " + genre + ": " + str(accuracy_score(real_labels_vect, prediction_vect)))
        print("Precision for " + genre + ": " + str(precision_score(real_labels_vect, prediction_vect)))
        print("Recall for " + genre + ": " + str(recall_score(real_labels_vect, prediction_vect)))
        print()

In [105]:
#size of intersection of predicted and actual labels divided by size of their union for each datapoint tested on
#sum those and then divide by number of datapoints
#vectorized for speed
def multi_label_accuracy(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    union = real_labels_matrix | predictions_labels_matrix
    #sum(array.T) gets number of 1s in row
    row_wise_accuracy = sum(intersection.T) / sum(union.T)
    return sum(row_wise_accuracy) / real_labels_matrix.shape[0]

#size of intersection of predicted and actual labels divided by size of predicted set for each datapoint tested on
#sum those and divide by number of datapoints
#if no predicted labels, don't count that row towards the precision as that would be undefined
def multi_label_precision(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    precision_sum = 0
    num_rows = 0
    for row in range(intersection.shape[0]):
        if sum(predictions_labels_matrix[row]) > 0: #if there is at least one prediction for this row
            num_rows += 1
            precision_sum += sum(intersection[row]) / sum(predictions_labels_matrix[row])
    if num_rows == 0:
        return 0#no labels predicted at all will give us 0 precision as precision makes no sense here
    return precision_sum / num_rows

#size of intersection of predicted and actual labels divided by size of real label set for each datapoint tested on
#sum those and divide by number of datapoints
#all datapoints should have at least 1 real label in this data set
#vectorized for speed
def multi_label_recall(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    #sum(array.T) gets number of 1s in row
    row_wise_recall = sum(intersection.T) / sum(real_labels_matrix.T)
    return sum(row_wise_recall) / real_labels_matrix.shape[0]

#lower is better
def hamming_loss(real_labels_matrix, predictions_labels_matrix):
    return (np.logical_xor(real_labels_matrix, predictions_labels_matrix)).sum()/(real_labels_matrix.shape[0] * real_labels_matrix.shape[1])

import keras.backend as K

#metric for keras for early stopping
#takes in raw labels from kerass (not yet converted to 0 and 1s)
#NOT the same as accuracy, this is total labels correctly identified divided by union of total labels
#this weights rows with more labels higher, where accruacy does not, but this is still a good metric for early stopping
def raw_multi_label_accuracy(y_true, y_pred):
    positives = K.greater_equal(y_pred, 0.5)
    positives = K.cast(positives, K.floatx())
    new_y_pred = positives #+ ((1-positives)*y_pred)
    intersection = y_true * new_y_pred
    union = 1 -((1-y_true)*(1-new_y_pred))
    accuracy = K.sum(intersection) / K.sum(union)
    return accuracy
    

In [106]:
from keras.callbacks import EarlyStopping
#for early stopping only after certain number of epochs. wait until delay epochs until early stopping
class DelayedEarlyStopping(EarlyStopping):
    def __init__(self, monitor, min_delta=0, patience=0, verbose=0, mode='auto', delay = 100):
        super(DelayedEarlyStopping, self).__init__()
        self.delay = delay

    def on_epoch_end(self, epoch, logs=None):
        if epoch > self.delay:
            super().on_epoch_end(epoch, logs)

In [107]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.regularizers import l2

model_cnn = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
#e = Embedding(num_words_kept, word_vec_len, input_length=max_seq_len, trainable=True)
model_cnn.add(e)
model_cnn.add(Conv1D(filters=50, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model_cnn.add(Dropout(.5))
model_cnn.add(Dense(50, activation='relu', kernel_regularizer=l2(0.01)))
model_cnn.add(Dropout(.5))
model_cnn.add(Dense(len(genre_dict), activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
#model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)
model_cnn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=200)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 4s - loss: 2.0995 - raw_multi_label_accuracy: 0.1724 - val_loss: 1.7789 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 1s - loss: 1.6067 - raw_multi_label_accuracy: 0.1151 - val_loss: 1.3835 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 2s - loss: 1.2630 - raw_multi_label_accuracy: 0.1171 - val_loss: 1.1084 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 2s - loss: 1.0326 - raw_multi_label_accuracy: 0.0857 - val_loss: 0.9232 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 2s - loss: 0.8762 - raw_multi_label_accuracy: 0.0666 - val_loss: 0.8015 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 6/1000
 - 2s - loss: 0.7745 - raw_multi_label_accuracy: 0.0573 - val_loss: 0.7227 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 7/1000
 - 2s - loss: 0.7071 - raw_multi_label_accuracy: 0.0460 - val_loss: 0.6706 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 8/1000
 - 2s - loss: 0.6650 - 

Epoch 65/1000
 - 1s - loss: 0.2689 - raw_multi_label_accuracy: 0.6925 - val_loss: 0.9176 - val_raw_multi_label_accuracy: 0.2829
Epoch 66/1000
 - 1s - loss: 0.2657 - raw_multi_label_accuracy: 0.7074 - val_loss: 0.8942 - val_raw_multi_label_accuracy: 0.2702
Epoch 67/1000
 - 2s - loss: 0.2631 - raw_multi_label_accuracy: 0.7057 - val_loss: 0.9038 - val_raw_multi_label_accuracy: 0.2778
Epoch 68/1000
 - 2s - loss: 0.2630 - raw_multi_label_accuracy: 0.7140 - val_loss: 0.9165 - val_raw_multi_label_accuracy: 0.2724
Epoch 69/1000
 - 2s - loss: 0.2616 - raw_multi_label_accuracy: 0.7094 - val_loss: 0.9141 - val_raw_multi_label_accuracy: 0.2775
Epoch 70/1000
 - 1s - loss: 0.2587 - raw_multi_label_accuracy: 0.7075 - val_loss: 0.9451 - val_raw_multi_label_accuracy: 0.2649
Epoch 71/1000
 - 2s - loss: 0.2568 - raw_multi_label_accuracy: 0.7180 - val_loss: 0.8904 - val_raw_multi_label_accuracy: 0.2760
Epoch 72/1000
 - 2s - loss: 0.2551 - raw_multi_label_accuracy: 0.7214 - val_loss: 0.9749 - val_raw_multi

Epoch 129/1000
 - 2s - loss: 0.1945 - raw_multi_label_accuracy: 0.7832 - val_loss: 1.2242 - val_raw_multi_label_accuracy: 0.2714
Epoch 130/1000
 - 1s - loss: 0.1926 - raw_multi_label_accuracy: 0.7861 - val_loss: 1.2263 - val_raw_multi_label_accuracy: 0.2771
Epoch 131/1000
 - 1s - loss: 0.1949 - raw_multi_label_accuracy: 0.7812 - val_loss: 1.2372 - val_raw_multi_label_accuracy: 0.2847
Epoch 132/1000
 - 1s - loss: 0.1946 - raw_multi_label_accuracy: 0.7791 - val_loss: 1.2212 - val_raw_multi_label_accuracy: 0.2941
Epoch 133/1000
 - 1s - loss: 0.1940 - raw_multi_label_accuracy: 0.7835 - val_loss: 1.2755 - val_raw_multi_label_accuracy: 0.2794
Epoch 134/1000
 - 2s - loss: 0.1926 - raw_multi_label_accuracy: 0.7855 - val_loss: 1.3151 - val_raw_multi_label_accuracy: 0.2785
Epoch 135/1000
 - 1s - loss: 0.1938 - raw_multi_label_accuracy: 0.7880 - val_loss: 1.1944 - val_raw_multi_label_accuracy: 0.2929
Epoch 136/1000
 - 1s - loss: 0.1902 - raw_multi_label_accuracy: 0.7866 - val_loss: 1.2296 - val_r

Epoch 193/1000
 - 1s - loss: 0.1686 - raw_multi_label_accuracy: 0.8203 - val_loss: 1.4048 - val_raw_multi_label_accuracy: 0.2814
Epoch 194/1000
 - 1s - loss: 0.1686 - raw_multi_label_accuracy: 0.8213 - val_loss: 1.3168 - val_raw_multi_label_accuracy: 0.3007
Epoch 195/1000
 - 1s - loss: 0.1660 - raw_multi_label_accuracy: 0.8194 - val_loss: 1.3244 - val_raw_multi_label_accuracy: 0.2947
Epoch 196/1000
 - 2s - loss: 0.1662 - raw_multi_label_accuracy: 0.8267 - val_loss: 1.3243 - val_raw_multi_label_accuracy: 0.2979
Epoch 197/1000
 - 1s - loss: 0.1645 - raw_multi_label_accuracy: 0.8224 - val_loss: 1.3643 - val_raw_multi_label_accuracy: 0.2962
Epoch 198/1000
 - 1s - loss: 0.1634 - raw_multi_label_accuracy: 0.8225 - val_loss: 1.4092 - val_raw_multi_label_accuracy: 0.3021
Epoch 199/1000
 - 1s - loss: 0.1609 - raw_multi_label_accuracy: 0.8157 - val_loss: 1.4711 - val_raw_multi_label_accuracy: 0.2796
Epoch 200/1000
 - 1s - loss: 0.1665 - raw_multi_label_accuracy: 0.8165 - val_loss: 1.3603 - val_r

In [108]:
def nn_output_to_predictions(res):
    label_predictions = []
    for i in range(res.shape[0]):
        pred = [0]*len(genre_dict)
        for j in range(res.shape[1]):
            if res[i][j] >= .5:
                pred[j] = 1
        label_predictions.append(pred)
    return np.array(label_predictions)

In [109]:
predictions = nn_output_to_predictions(model_cnn.predict(x_test_seq))

In [110]:
y_test[:,0].sum()

187

In [111]:
predictions[:,0].sum()

273

In [112]:
predictions[0]

array([1, 0, 1, 0, 0])

In [113]:
multi_label_accuracy(y_test, predictions)

0.36849551414768833

In [114]:
multi_label_precision(y_test, predictions)

0.4861702127659575

In [115]:
multi_label_recall(y_test, predictions)

0.4801587301587302

In [116]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 66.45962732919256


In [117]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.556935817805383
Precision for Action-Adventure: 0.45054945054945056
Recall for Action-Adventure: 0.6577540106951871

Accuruacy for Romance: 0.7619047619047619
Precision for Romance: 0.4117647058823529
Recall for Romance: 0.19811320754716982

Accuruacy for Horror-Thriller: 0.5755693581780539
Precision for Horror-Thriller: 0.4723404255319149
Recall for Horror-Thriller: 0.578125

Accuruacy for Comedy: 0.5942028985507246
Precision for Comedy: 0.5521472392638037
Recall for Comedy: 0.4225352112676056

Accuruacy for Science Fiction: 0.8343685300207039
Precision for Science Fiction: 0.2
Recall for Science Fiction: 0.14285714285714285



CNN but with multiple filter sizes so we don't just filter on group of words at a time

In [120]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model

model_input = Input(shape=(max_seq_len,), dtype='int32')

e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)(model_input)
two_word_filter = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(e)
two_word_filter = GlobalMaxPooling1D()(two_word_filter)
three_word_filter = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(e)
three_word_filter = GlobalMaxPooling1D()(three_word_filter)
four_word_filter = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(e)
four_word_filter = GlobalMaxPooling1D()(four_word_filter)
merged = concatenate([two_word_filter, three_word_filter, four_word_filter], axis=1)

merged = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(merged)
merged = Dropout(0.5)(merged)
merged = Dense(len(genre_dict))(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[model_input], outputs=[output])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])

In [121]:
model.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 5s - loss: 2.8998 - raw_multi_label_accuracy: 0.0729 - val_loss: 2.3065 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 4s - loss: 1.9396 - raw_multi_label_accuracy: 0.0403 - val_loss: 1.5543 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 4s - loss: 1.3356 - raw_multi_label_accuracy: 0.0406 - val_loss: 1.1050 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 4s - loss: 0.9876 - raw_multi_label_accuracy: 0.0147 - val_loss: 0.8601 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 4s - loss: 0.7961 - raw_multi_label_accuracy: 0.0416 - val_loss: 0.7256 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 6/1000
 - 4s - loss: 0.6933 - raw_multi_label_accuracy: 0.0333 - val_loss: 0.6593 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 7/1000
 - 4s - loss: 0.6398 - raw_multi_label_accuracy: 0.0203 - val_loss: 0.6254 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 8/1000
 - 5s - loss: 0.6034 - 

In [122]:
predictions = nn_output_to_predictions(model.predict(x_test_seq))

In [123]:
multi_label_accuracy(y_test, predictions)

0.3887163561076607

In [124]:
multi_label_precision(y_test, predictions)

0.5672782874617737

In [125]:
multi_label_recall(y_test, predictions)

0.47066942719116645

In [126]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 71.01449275362319


In [127]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.639751552795031
Precision for Action-Adventure: 0.536723163841808
Recall for Action-Adventure: 0.5080213903743316

Accuruacy for Romance: 0.7349896480331263
Precision for Romance: 0.13333333333333333
Recall for Romance: 0.03773584905660377

Accuruacy for Horror-Thriller: 0.6521739130434783
Precision for Horror-Thriller: 0.5631578947368421
Recall for Horror-Thriller: 0.5572916666666666

Accuruacy for Comedy: 0.6418219461697723
Precision for Comedy: 0.5909090909090909
Recall for Comedy: 0.6103286384976526

Accuruacy for Science Fiction: 0.8819875776397516
Precision for Science Fiction: 0.3333333333333333
Recall for Science Fiction: 0.017857142857142856



Regular Neural Network

In [128]:
normal_nn = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
normal_nn.add(e)
normal_nn.add(Flatten())
normal_nn.add(Dense(256, activation='relu'))
normal_nn.add(Dense(len(genre_dict), activation='sigmoid'))
normal_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
normal_nn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 3s - loss: 0.6201 - raw_multi_label_accuracy: 0.0391 - val_loss: 0.5979 - val_raw_multi_label_accuracy: 0.0572
Epoch 2/1000
 - 1s - loss: 0.5824 - raw_multi_label_accuracy: 0.0770 - val_loss: 0.5973 - val_raw_multi_label_accuracy: 0.1666
Epoch 3/1000
 - 2s - loss: 0.5579 - raw_multi_label_accuracy: 0.1495 - val_loss: 0.5978 - val_raw_multi_label_accuracy: 0.1513
Epoch 4/1000
 - 2s - loss: 0.5283 - raw_multi_label_accuracy: 0.2400 - val_loss: 0.5934 - val_raw_multi_label_accuracy: 0.1645
Epoch 5/1000
 - 2s - loss: 0.4755 - raw_multi_label_accuracy: 0.3542 - val_loss: 0.6018 - val_raw_multi_label_accuracy: 0.1652
Epoch 6/1000
 - 2s - loss: 0.3837 - raw_multi_label_accuracy: 0.5507 - val_loss: 0.5813 - val_raw_multi_label_accuracy: 0.2200
Epoch 7/1000
 - 2s - loss: 0.2795 - raw_multi_label_accuracy: 0.7035 - val_loss: 0.5818 - val_raw_multi_label_accuracy: 0.2336
Epoch 8/1000
 - 1s - loss: 0.1952 - raw_multi_label_accuracy: 0.

In [129]:
predictions = nn_output_to_predictions(normal_nn.predict(x_test_seq))

In [130]:
multi_label_accuracy(y_test, predictions)

0.3064182194616977

In [131]:
multi_label_precision(y_test, predictions)

0.5862068965517241

In [132]:
multi_label_recall(y_test, predictions)

0.3309178743961353

In [133]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 71.7184265010352


In [134]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.7163561076604554
Precision for Action-Adventure: 0.7232142857142857
Recall for Action-Adventure: 0.43315508021390375

Accuruacy for Romance: 0.782608695652174
Precision for Romance: 0.5172413793103449
Recall for Romance: 0.14150943396226415

Accuruacy for Horror-Thriller: 0.6252587991718427
Precision for Horror-Thriller: 0.5523809523809524
Recall for Horror-Thriller: 0.3020833333333333

Accuruacy for Comedy: 0.5714285714285714
Precision for Comedy: 0.5182926829268293
Recall for Comedy: 0.39906103286384975

Accuruacy for Science Fiction: 0.8902691511387164
Precision for Science Fiction: 0.8
Recall for Science Fiction: 0.07142857142857142



LSTM

In [135]:
from keras.layers import LSTM
lstm_model = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
lstm_model.add(e)
lstm_model.add(LSTM(100, dropout=0.25, recurrent_dropout=0.25))
lstm_model.add(Dense(len(genre_dict), activation='sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
lstm_model.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 11s - loss: 0.6398 - raw_multi_label_accuracy: 0.0461 - val_loss: 0.5898 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 9s - loss: 0.5956 - raw_multi_label_accuracy: 0.0464 - val_loss: 0.5866 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 7s - loss: 0.5878 - raw_multi_label_accuracy: 0.0000e+00 - val_loss: 0.5814 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 7s - loss: 0.5844 - raw_multi_label_accuracy: 0.0022 - val_loss: 0.5779 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 7s - loss: 0.5803 - raw_multi_label_accuracy: 0.0000e+00 - val_loss: 0.5773 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 6/1000
 - 7s - loss: 0.5717 - raw_multi_label_accuracy: 0.0000e+00 - val_loss: 0.5721 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 7/1000
 - 7s - loss: 0.5478 - raw_multi_label_accuracy: 0.1440 - val_loss: 0.5436 - val_raw_multi_label_accuracy: 0.1666
Epoch 8/1000
 - 8s - loss: 

In [136]:
predictions = nn_output_to_predictions(lstm_model.predict(x_test_seq))

In [137]:
multi_label_accuracy(y_test, predictions)

0.4903381642512081

In [138]:
multi_label_precision(y_test, predictions)

0.6324965132496515

In [139]:
multi_label_recall(y_test, predictions)

0.5876466528640443

In [140]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 75.03105590062111


In [141]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.6873706004140787
Precision for Action-Adventure: 0.62
Recall for Action-Adventure: 0.49732620320855614

Accuruacy for Romance: 0.8012422360248447
Precision for Romance: 0.5806451612903226
Recall for Romance: 0.33962264150943394

Accuruacy for Horror-Thriller: 0.660455486542443
Precision for Horror-Thriller: 0.5752688172043011
Recall for Horror-Thriller: 0.5572916666666666

Accuruacy for Comedy: 0.7080745341614907
Precision for Comedy: 0.6428571428571429
Recall for Comedy: 0.7605633802816901

Accuruacy for Science Fiction: 0.8944099378881988
Precision for Science Fiction: 0.6086956521739131
Recall for Science Fiction: 0.25

